In [ ]:
from elasticsearch import Elasticsearch
import os
import json
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
import re
def iterate_val(dictionary):
    for item in dictionary:
        if item == '' or item == [] or item == {}:
            if dictionary[item] == [] or dictionary[item] == '' or dictionary[item] == {}:
                dictionary[item] = ["0"]
            dictionary["Null Value Found"] = dictionary[item]
            del dictionary[item]
            continue
        if dictionary[item] == {Ellipsis}:
            dictionary[item] = {"Null Value Here" : "0"}
        if isinstance(dictionary[item],dict):
            iterate_val(dictionary[item])
        if isinstance(dictionary[item],list):
            for idx in dictionary[item]:
                if isinstance(idx,dict):
                    iterate_val(idx)
    return dictionary
# contents_clean = iterate_val(contents)

In [ ]:
'''To index all cases into a dictionary'''
path_to_parent = ""
count = 0
cite_dict = []
for court in tqdm(os.listdir(path_to_parent)):
    if court.endswith('.json') or court.endswith('.pdf') or court.endswith('.txt') or court.endswith('.jpg'):
            continue
    for year in tqdm(os.listdir(os.path.join(path_to_parent))): 
        if year.endswith('.json') or year.endswith('.pdf') or year.endswith('.txt') or year.endswith('.jpg'):
            continue
        for month in os.listdir(os.path.join(path_to_parent, year)):
            if month.endswith('.json') or month.endswith('.pdf') or month.endswith('.txt') or month.endswith('.jpg'):
                continue
            for files in os.listdir(os.path.join(path_to_parent, year, month)):
                if files.endswith('.json'):
                    x = str(os.path.join(path_to_parent, year,month,files))
                    with open(x, 'r') as j:
                        contents = json.loads(j.read())
    #                     print(len(contents))
                        contents['cites'] = str(contents['cites'])
                        contents['citedBy'] = str(contents['citedBy'])
                        for i in range(0,len(contents['paragraphs'])):
                            contents['paragraphs'][i]['mainPara'] = str(contents['paragraphs'][i]['mainPara'])
                            contents['paragraphs'][i]['subPara'] = str(contents['paragraphs'][i]['subPara'])
                        contents_clean = iterate_val(contents)
    #                     print(len(contents_clean))
                        cite_dict.append(contents_clean)

In [ ]:
new_lst = []
for i in tqdm(range(0,len(cite_dict))):
    index = {"index":{"_index":"cases","_id":str(i)}}
    new_lst.append(index)
    new_lst.append(cite_dict[i])
len(new_lst)
new_low_lst = new_lst[0:1500]

In [ ]:
with open('output_cases.json', 'w') as f:
    for d in new_lst:
        string = json.dumps(d)
        f.write(string+'\n')

In [ ]:
lst = [new_low_lst[i:i+50] for i in range(0, len(new_low_lst), 50)]

In [ ]:
import requests
for item in lst:
    with open('output_cases.json', 'w') as f:
        for d in item:
            string = json.dumps(d)
            f.write(string+'\n')
    headers = {
    'Content-Type': 'application/json',}
    data = open('output_cases.json', 'rb').read()
    response = requests.post('https://search-lawnics-h6i3e4ys7i26faaqoqsmqoew3m.us-east-2.es.amazonaws.com//_bulk', 
                             headers=headers, data=data, auth=('administrator', 'L@wnics2020'))
    print("done")